# Data Manipulation Scenario


## Scenario

The network consists of an office subnet and a server subnet. Clients in the office access a website which fetches data from a database.

[<img src="_package_data/uc2_network.png" width="500"/>](_package_data/uc2_network.png)

_(click image to enlarge)_

The red agent deletes the contents of the database. When this happens, the web app cannot fetch data and users navigating to the website get a 404 error.


## Network

- The web server has:
    - a web service that replies to user HTTP requests
    - a database client that fetches data for the web service
- The database server has:
    - a POSTGRES database service
    - a database file which is accessed by the database service
    - FTP client used for backing up the data to the backup_server
- The backup server has:
    - a copy of the database file in a known good state
    - FTP server that can send the backed up file back to the database server


## Green agent

There are green agents logged onto client 1 and client 2. They use the web browser to navigate to `http://arcd.com/users`. The web server replies with a status code 200 if the data is available on the database or 404 if not available.

## Red agent

At the start of every episode, the red agent randomly chooses either client 1 or client 2 to login to. It waits a bit then sends a DELETE query to the database from its chosen client. If the delete is successful, the database file is flagged as compromised to signal that data is not available.

[<img src="_package_data/uc2_attack.png" width="500"/>](_package_data/uc2_attack.png)

_(click image to enlarge)_

## Blue agent

The blue agent can view the entire network, but the health statuses of components are not updated until a scan is performed. The blue agent should restore the database file from backup after it was compromised. It can also prevent further attacks by blocking the red agent client from sending the malicious SQL query to the database server. This can be done by implementing an ACL rule on the router.

# Reinforcement learning details

## Scripted agents:
### Red
The red agent sits on a client and uses an application called DataManipulationBot whose sole purpose is to send a DELETE query to the database.
The red agent can choose one of two action each timestep:
1. do nothing
2. execute the data manipulation application
The schedule for selecting when to execute the application is controlled by three parameters:
- start time
- frequency
- variance

Attacks start at a random timestep between (start_time - variance) and (start_time + variance). After each attack, another is attempted after a random delay between (frequency - variance) and (frequency + variance) timesteps.

The data manipulation app itself has an element of randomness because the attack has a probability of success. The default is 0.8 to succeed with the port scan step and 0.8 to succeed with the attack itself.
Upon a successful attack, the database file becomes corrupted which incurs a negative reward for the RL defender.

The red agent does not use information about the state of the network to decide its action.

### Green
The green agents use the web browser application to send requests to the web server. The schedule of each green agent is currently random, meaning it will request webpage with a 50% probability, and do nothing with a 50% probability.

When a green agent is blocked from accessing the data through the webpage, this incurs a negative reward to the RL defender.

## Observation Space

The blue agent's observation space is structured as nested dictionary with the following information:
```

- NODES
    - <node_id 1-7>
        - SERVICES
            - <service_id 1-1>
                - operating_status
                - health_status
        - FOLDERS
            - <folder_id 1-1>
                - health_status
                - FILES
                    - <file_id 1-1>
                    - health_status
        - NICS
            - <nic_id 1-2>
                - nic_status
        - operating_status
- LINKS
    - <link_id 1-10>
        - PROTOCOLS
            - ALL
                - load
- ACL
    - <rule_number 1-10>
        - position
        - permission
        - source_node_id
        - source_port
        - dest_node_id
        - dest_port
        - protocol
- ICS
```

### Mappings

The dict keys for `node_id` are in the following order:
|node_id|node name|
|--|--|
|1|domain_controller|
|2|web_server|
|3|database_server|
|4|backup_server|
|5|security_suite|
|6|client_1|
|7|client_2|

Service 1 on node 2 (web_server) corresponds to the Web Server service. Other services are only there for padding to ensure that each node's observation space has the same shape. They are filled with zeroes.

Folder 1 on node 3 corresponds to the database folder. File 1 in that folder corresponds to the database storage file. Other files and folders are only there for padding to ensure that each node's observation space has the same shape. They are filled with zeroes.

The dict keys for `link_id` are in the following order:
|link_id|endpoint_a|endpoint_b|
|--|--|--|
|1|router_1|switch_1|
|1|router_1|switch_2|
|1|switch_1|domain_controller|
|1|switch_1|web_server|
|1|switch_1|database_server|
|1|switch_1|backup_server|
|1|switch_1|security_suite|
|1|switch_2|client_1|
|1|switch_2|client_2|
|1|switch_2|security_suite|

The ACL rules in the observation space appear in the same order that they do in the actual ACL. Though, only the first 10 rules are shown, there are default rules lower down that cannot be changed by the agent. The extra rules just allow the network to function normally, by allowing pings, ARP traffic, etc.

Most nodes have only 1 nic, so the observation for those is placed at NIC index 1 in the observation space. Only the security suite has 2 NICs, the second NIC in the observation space is the one that connects the security suite with swtich_2.

The meaning of the services' operating_state is:
|operating_state|label|
|--|--|
|0|UNUSED|
|1|RUNNING|
|2|STOPPED|
|3|PAUSED|
|4|DISABLED|
|5|INSTALLING|
|6|RESTARTING|

The meaning of the services' health_state is:
|health_state|label|
|--|--|
|0|UNUSED|
|1|GOOD|
|2|PATCHING|
|3|COMPROMISED|
|4|OVERWHELMED|

The meaning of the files' and folders' health_state is:
|health_state|label|
|--|--|
|0|UNUSED|
|1|GOOD|
|2|COMPROMISED|
|3|CORRUPT|
|4|RESTORING|
|5|REPAIRING|

The meaning of the NICs' operating_status is:
|operating_status|label|
|--|--|
|0|UNUSED|
|1|ENABLED|
|2|DISABLED|

Link load has the following meaning:
|load|percent utilisation|
|--|--|
|0|exactly 0%|
|1|0-11%|
|2|11-22%|
|3|22-33%|
|4|33-44%|
|5|44-55%|
|6|55-66%|
|7|66-77%|
|8|77-88%|
|9|88-99%|
|10|exactly 100%|

ACL permission has the following meaning:
|permission|label|
|--|--|
|0|UNUSED|
|1|ALLOW|
|2|DENY|

ACL source / destination node ids actually correspond to IP addresses (since ACLs work with IP addresses)
|source / dest node id|ip_address|label|
|--|--|--|
|0| | UNUSED|
|1| |ALL addresses|
|2| 192.168.1.10  | domain_controller|
|3| 192.168.1.12  | web_server 
|4| 192.168.1.14  | database_server|
|5| 192.168.1.16  | backup_server|
|6| 192.168.1.110 | security_suite (eth-1)|
|7| 192.168.10.21 | client_1|
|8| 192.168.10.22 | client_2|
|9| 192.168.10.110| security_suite (eth-2)|

ACL source / destination port ids have the following encoding:
|port id|port number| port use |
|--|--|--|
|0||UNUSED|
|1||ALL|
|2|219|ARP|
|3|53|DNS|
|4|80|HTTP|
|5|5432|POSTGRES_SERVER|

ACL protocol ids have the following encoding:
|protocol id|label|
|--|--|
|0|UNUSED|
|1|ALL|
|2|ICMP|
|3|TCP|
|4|UDP|

protocol

## Action Space

The blue agent chooses from a list of 54 pre-defined actions. The full list is defined in the `action_map` in the config. The most important ones are explained here:

- `0`: Do nothing
- `1`: Scan the web service - this refreshes the health status in the observation space
- `9`: Scan the database file - this refreshes the health status of the database file
- `13`: Patch the database service - This triggers the database to restore data from the backup server
- `19`: Shut down client 1
- `20`: Start up client 1
- `22`: Block outgoing traffic from client 1
- `23`: Block outgoing traffic from client 2
- `26`: Block TCP traffic from client 1 to the database node
- `27`: Block TCP traffic from client 2 to the database node
- `28-37`: Remove ACL rules 1-10
- `42`: Disconnect client 1 from the network
- `43`: Reconnect client 1 to the network
- `44`: Disconnect client 2 from the network
- `45`: Reconnect client 2 to the network

The other actions will either have no effect or will negatively impact the network, so the blue agent should avoid taking them.

## Reward Function

The blue agent's reward is calculated using two measures:
1. Whether the database file is in a good state (+1 for good, -1 for corrupted, 0 for any other state)
2. Whether each green agents' most recent webpage request was successful (+1 for a `200` return code, -1 for a `404` return code and 0 otherwise).
The file status reward and the two green-agent-related reward are averaged to get a total step reward.


## Demonstration

First, load the required modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
from primaite.config.load import example_config_path
from primaite.session.environment import PrimaiteGymEnv
from primaite.game.game import PrimaiteGame
import yaml
from pprint import pprint


/home/cade/repos/PrimAITE/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-07 10:58:13,192	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-07 10:58:17,136	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Instantiate the environment. We also disable the agent observation flattening.

This cell will print the observation when the network is healthy. You should be able to verify Node file and service statuses against the description above.

In [3]:
# create the env
with open(example_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)
    # set success probability to 1.0 to avoid rerunning cells.
    cfg['simulation']['network']['nodes'][8]['applications'][0]['options']['data_manipulation_p_of_success'] = 1.0
    cfg['simulation']['network']['nodes'][9]['applications'][0]['options']['data_manipulation_p_of_success'] = 1.0
    cfg['simulation']['network']['nodes'][8]['applications'][0]['options']['port_scan_p_of_success'] = 1.0
    cfg['simulation']['network']['nodes'][9]['applications'][0]['options']['port_scan_p_of_success'] = 1.0
game = PrimaiteGame.from_config(cfg)
env = PrimaiteGymEnv(game = game)
# Don't flatten obs as we are not training an agent and we wish to see the dict-formatted observations
env.agent.flatten_obs = False
obs, info = env.reset()
print('env created successfully')
pprint(obs)

Resetting environment, episode 0, avg. reward: 0.0
env created successfully
{'ACL': {1: {'dest_node_id': 0,
             'dest_port': 0,
             'permission': 0,
             'position': 0,
             'protocol': 0,
             'source_node_id': 0,
             'source_port': 0},
         2: {'dest_node_id': 0,
             'dest_port': 0,
             'permission': 0,
             'position': 1,
             'protocol': 0,
             'source_node_id': 0,
             'source_port': 0},
         3: {'dest_node_id': 0,
             'dest_port': 0,
             'permission': 0,
             'position': 2,
             'protocol': 0,
             'source_node_id': 0,
             'source_port': 0},
         4: {'dest_node_id': 0,
             'dest_port': 0,
             'permission': 0,
             'position': 3,
             'protocol': 0,
             'source_node_id': 0,
             'source_port': 0},
         5: {'dest_node_id': 0,
             'dest_port': 0,
           

The red agent will start attacking at some point between step 20 and 30. When this happens, the reward will drop immediately, then drop to -1.0 when green agents try to access the webpage.

In [4]:
def friendly_output_red_action(info):
    # parse the info dict form step output and write out what the red agent is doing
    red_info = info['agent_actions']['data_manipulation_attacker']
    red_action = red_info[0]
    if red_action == 'DONOTHING':
        red_str = 'DO NOTHING'
    elif red_action == 'NODE_APPLICATION_EXECUTE':
        client = "client 1" if red_info[1]['node_id'] == 0 else "client 2"
        red_str = f"ATTACK from {client}"
    return red_str

In [5]:
for step in range(35):
    obs, reward, terminated, truncated, info = env.step(0)
    print(f"step: {env.game.step_counter}, Red action: {friendly_output_red_action(info)}, Blue reward:{reward}" )

step: 1, Red action: DO NOTHING, Blue reward:0.34
step: 2, Red action: DO NOTHING, Blue reward:1.0
step: 3, Red action: DO NOTHING, Blue reward:1.0
step: 4, Red action: DO NOTHING, Blue reward:1.0
step: 5, Red action: DO NOTHING, Blue reward:1.0
step: 6, Red action: DO NOTHING, Blue reward:1.0
step: 7, Red action: DO NOTHING, Blue reward:1.0
step: 8, Red action: DO NOTHING, Blue reward:1.0
step: 9, Red action: DO NOTHING, Blue reward:1.0
step: 10, Red action: DO NOTHING, Blue reward:1.0
step: 11, Red action: DO NOTHING, Blue reward:1.0
step: 12, Red action: DO NOTHING, Blue reward:1.0
step: 13, Red action: DO NOTHING, Blue reward:1.0
step: 14, Red action: DO NOTHING, Blue reward:1.0
step: 15, Red action: DO NOTHING, Blue reward:1.0
step: 16, Red action: DO NOTHING, Blue reward:1.0
step: 17, Red action: DO NOTHING, Blue reward:1.0
step: 18, Red action: DO NOTHING, Blue reward:1.0
step: 19, Red action: DO NOTHING, Blue reward:1.0
step: 20, Red action: DO NOTHING, Blue reward:1.0
step: 21

Now the reward is -1, let's have a look at blue agent's observation.

In [6]:
pprint(obs['NODES'])

{1: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health_status': 0}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 0, 'operating_status': 1}},
     'operating_status': 1},
 2: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health_status': 0}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 0, 'operating_status': 1}},
     'operating_status': 1},
 3: {'FOLDERS': {1: {'FILES': {1: {'health_status': 1}}, 'health_status': 1}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 0, 'operating_status': 0}},
     'operating_status': 1},
 4: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health_status': 0}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 0, 'operating_status': 0}},
     'operating_status': 1},
 5: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health

The true statuses of the database file and webapp are not updated. The blue agent needs to perform a scan to see that they have degraded.

In [7]:
obs, reward, terminated, truncated, info = env.step(9)  # scan database file
obs, reward, terminated, truncated, info = env.step(1)  # scan webapp service
pprint(obs['NODES'])

{1: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health_status': 0}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 0, 'operating_status': 1}},
     'operating_status': 1},
 2: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health_status': 0}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 3, 'operating_status': 1}},
     'operating_status': 1},
 3: {'FOLDERS': {1: {'FILES': {1: {'health_status': 2}}, 'health_status': 1}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 0, 'operating_status': 0}},
     'operating_status': 1},
 4: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health_status': 0}},
     'NICS': {1: {'nic_status': 1}, 2: {'nic_status': 0}},
     'SERVICES': {1: {'health_status': 0, 'operating_status': 0}},
     'operating_status': 1},
 5: {'FOLDERS': {1: {'FILES': {1: {'health_status': 0}}, 'health

Now service 1 on node 2 has `health_status = 3`, indicating that the webapp is compromised.
File 1 in folder 1 on node 3 has `health_status = 2`, indicating that the database file is compromised.

The blue agent can now patch the database to restore the file to a good health status.

In [8]:
obs, reward, terminated, truncated, info = env.step(13)  # patch the database
print(f"step: {env.game.step_counter}")
print(f"Red action: {info['agent_actions']['data_manipulation_attacker'][0]}" )
print(f"Green action: {info['agent_actions']['client_1_green_user'][0]}" )
print(f"Green action: {info['agent_actions']['client_2_green_user'][0]}" )
print(f"Blue reward:{reward}" )

step: 38
Red action: DONOTHING
Green action: NODE_APPLICATION_EXECUTE
Green action: NODE_APPLICATION_EXECUTE
Blue reward:-1.0


The patching takes two steps, so the reward hasn't changed yet. Let's do nothing for another timestep, the reward should improve.

The reward will increase slightly as soon as the file finishes restoring. Then, the reward will increase to 1 when both green agents make successful requests.

Run the following cell until the green action is `NODE_APPLICATION_EXECUTE`, then the reward should become 1. If you run it enough times, another red attack will happen and the reward will drop again.

In [9]:
obs, reward, terminated, truncated, info = env.step(0)  # patch the database
print(f"step: {env.game.step_counter}")
print(f"Red action: {info['agent_actions']['data_manipulation_attacker'][0]}" )
print(f"Green action: {info['agent_actions']['client_2_green_user'][0]}" )
print(f"Green action: {info['agent_actions']['client_1_green_user'][0]}" )
print(f"Blue reward:{reward}" )

step: 39
Red action: DONOTHING
Green action: NODE_APPLICATION_EXECUTE
Green action: DONOTHING
Blue reward:-0.32


The blue agent can prevent attacks by implementing an ACL rule to stop client_1 or client_2 from sending POSTGRES traffic to the database. (Let's also patch the database file to get the reward back up.)

Let's block both clients from communicating directly with the database.

In [17]:
env.step(13)  # Patch the database
print(f"step: {env.game.step_counter}, Red action: {info['agent_actions']['data_manipulation_attacker'][0]}, Blue reward:{reward}" )

env.step(26)  # Block client 1
print(f"step: {env.game.step_counter}, Red action: {info['agent_actions']['data_manipulation_attacker'][0]}, Blue reward:{reward}" )

env.step(27)  # Block client 2
print(f"step: {env.game.step_counter}, Red action: {info['agent_actions']['data_manipulation_attacker'][0]}, Blue reward:{reward}" )

for step in range(30):
    obs, reward, terminated, truncated, info = env.step(0)  # do nothing
    print(f"step: {env.game.step_counter}, Red action: {info['agent_actions']['data_manipulation_attacker'][0]}, Blue reward:{reward}" )

step: 139, Red action: DONOTHING, Blue reward:-0.32
step: 140, Red action: DONOTHING, Blue reward:-0.32
step: 141, Red action: DONOTHING, Blue reward:-0.32
step: 142, Red action: DONOTHING, Blue reward:-0.32
step: 143, Red action: DONOTHING, Blue reward:-0.32
step: 144, Red action: DONOTHING, Blue reward:-0.32
step: 145, Red action: DONOTHING, Blue reward:-0.32
step: 146, Red action: DONOTHING, Blue reward:-0.32
step: 147, Red action: DONOTHING, Blue reward:-0.32
step: 148, Red action: DONOTHING, Blue reward:-0.32
step: 149, Red action: NODE_APPLICATION_EXECUTE, Blue reward:-0.32
step: 150, Red action: DONOTHING, Blue reward:-0.32
step: 151, Red action: DONOTHING, Blue reward:-0.32
step: 152, Red action: DONOTHING, Blue reward:-0.32
step: 153, Red action: DONOTHING, Blue reward:-0.32
step: 154, Red action: DONOTHING, Blue reward:-0.32
step: 155, Red action: DONOTHING, Blue reward:-0.32
step: 156, Red action: DONOTHING, Blue reward:-0.32
step: 157, Red action: DONOTHING, Blue reward:-0.

Now, even though the red agent executes an attack, the reward stays at 1.0

Let's also have a look at the ACL observation to verify our new ACL rule at positions 5 and 6.

In [18]:
obs['ACL']

{1: {'position': 0,
  'permission': 0,
  'source_node_id': 0,
  'source_port': 0,
  'dest_node_id': 0,
  'dest_port': 0,
  'protocol': 0},
 2: {'position': 1,
  'permission': 0,
  'source_node_id': 0,
  'source_port': 0,
  'dest_node_id': 0,
  'dest_port': 0,
  'protocol': 0},
 3: {'position': 2,
  'permission': 0,
  'source_node_id': 0,
  'source_port': 0,
  'dest_node_id': 0,
  'dest_port': 0,
  'protocol': 0},
 4: {'position': 3,
  'permission': 0,
  'source_node_id': 0,
  'source_port': 0,
  'dest_node_id': 0,
  'dest_port': 0,
  'protocol': 0},
 5: {'position': 4,
  'permission': 2,
  'source_node_id': 7,
  'source_port': 1,
  'dest_node_id': 4,
  'dest_port': 1,
  'protocol': 3},
 6: {'position': 5,
  'permission': 2,
  'source_node_id': 8,
  'source_port': 1,
  'dest_node_id': 4,
  'dest_port': 1,
  'protocol': 3},
 7: {'position': 6,
  'permission': 0,
  'source_node_id': 0,
  'source_port': 0,
  'dest_node_id': 0,
  'dest_port': 0,
  'protocol': 0},
 8: {'position': 7,
  'perm

In [13]:
router = env.game.simulation.network.get_node_by_hostname('router_1')

In [19]:
router.acl.show()

+------------------------------------------------------------------------------------------------------------+
|                                        router_1 Access Control List                                        |
+-------+--------+----------+---------------+------------------------+--------------+------------------------+
| Index | Action | Protocol | Src IP        | Src Port               | Dst IP       | Dst Port               |
+-------+--------+----------+---------------+------------------------+--------------+------------------------+
| 5     | DENY   | TCP      | 192.168.10.21 | ANY                    | 192.168.1.14 | ANY                    |
| 6     | DENY   | TCP      | 192.168.10.22 | ANY                    | 192.168.1.14 | ANY                    |
| 18    | PERMIT | ANY      | ANY           | 5432 (POSTGRES_SERVER) | ANY          | 5432 (POSTGRES_SERVER) |
| 19    | PERMIT | ANY      | ANY           | 53 (DNS)               | ANY          | 53 (DNS)               |
|

In [22]:
env.agent.reward_function.reward_components

[(<primaite.game.agent.rewards.DatabaseFileIntegrity at 0x7f0d587b9e40>, 0.34),
 (<primaite.game.agent.rewards.WebpageUnavailablePenalty at 0x7f0d587ba0b0>,
  0.33),
 (<primaite.game.agent.rewards.WebpageUnavailablePenalty at 0x7f0d587ba170>,
  0.33)]

In [23]:
client_1 = env.game.simulation.network.get_node_by_hostname('client_1')
client_2 = env.game.simulation.network.get_node_by_hostname('client_2')

In [24]:
client_1_browser = client_1.software_manager.software.get("WebBrowser")
client_2_browser = client_2.software_manager.software.get("WebBrowser")

In [26]:
client_2_browser.history

[BrowserHistoryItem(url='http://arcd.com/users/', status=<_HistoryItemStatus.LOADED: 'LOADED'>, response_code=<HttpStatusCode.OK: 200>),
 BrowserHistoryItem(url='http://arcd.com/users/', status=<_HistoryItemStatus.LOADED: 'LOADED'>, response_code=<HttpStatusCode.OK: 200>),
 BrowserHistoryItem(url='http://arcd.com/users/', status=<_HistoryItemStatus.LOADED: 'LOADED'>, response_code=<HttpStatusCode.OK: 200>),
 BrowserHistoryItem(url='http://arcd.com/users/', status=<_HistoryItemStatus.LOADED: 'LOADED'>, response_code=<HttpStatusCode.OK: 200>),
 BrowserHistoryItem(url='http://arcd.com/users/', status=<_HistoryItemStatus.LOADED: 'LOADED'>, response_code=<HttpStatusCode.OK: 200>),
 BrowserHistoryItem(url='http://arcd.com/users/', status=<_HistoryItemStatus.LOADED: 'LOADED'>, response_code=<HttpStatusCode.OK: 200>),
 BrowserHistoryItem(url='http://arcd.com/users/', status=<_HistoryItemStatus.LOADED: 'LOADED'>, response_code=<HttpStatusCode.OK: 200>),
 BrowserHistoryItem(url='http://arcd.com/

In [35]:
client_1_browser.get_webpage()

False

In [30]:
database_server = env.game.simulation.network.get_node_by_hostname('database_server')

In [31]:
database_server.file_system.get_file('database', 'database.db')

File(uuid='390c399a-c2ab-4d84-b98f-d5fc1f9114d2', name='database.db', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.GOOD: 1>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x7f0d58908400>, deleted=False, folder_id='e602b9a6-ea1c-4c52-9025-8512d9116b1d', folder_name='database', file_type=<FileType.DB: 32>, sim_size=15360000, real=False, sim_path=None, sim_root=PosixPath('/home/cade/primaite/3.0.0b6/sessions/2024-02-07/10-58-18/simulation_output/database_server/fs'), folder=Folder(uuid='e602b9a6-ea1c-4c52-9025-8512d9116b1d', name='database', health_status=<FileSystemItemHealthStatus.GOOD: 1>, visible_health_status=<FileSystemItemHealthStatus.GOOD: 1>, previous_hash=None, revealed_to_red=False, sys_log=<primaite.simulator.system.core.sys_log.SysLog object at 0x7f0d58908400>, deleted=False, files={'390c399a-c2ab-4d84-b98f-d5fc1f9114d2': File(uuid='390c399a-c2ab-4d84-b98f-d5

In [ ]:
database_server.